In [2]:
using Pkg
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("Random")
using LinearAlgebra
using Plots
using Random

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


# Nice

## Neurons

In [3]:
function generate_memories(
        N,
        n
        )
    """
    Generate n random N neuron memories.
    Columns neuron states
    Rows memory
    """
    return bitrand((N,n))
end 

function initialise_weights(
        memories
        )
    """
    Initialise and return weight matrix.
    
    Given the matrix of memories to store where each column 
    is a state, each row a memory, return a matrix of
    weights between neurons in network via Hebbian 
    learning prescription.
    """
    # Initialise Tij weight matrix 
    N = length(memories[:,1])
    n = length(memories[1,:])
    T = zeros(
        Int8,
        (N,N)
    )
    
    # Iterate via Hebbian learning rule
    for ii in 1:N
        for jj in 1:N
            for kk in 1:n
                T[ii,jj] += (2*memories[ii,kk] - 1)*(2*memories[jj,kk]-1)
            end
        end
    end         
    return T
end

function initialise_network(
    N
)
"""
Initialise and return network of neurons.
"""
return bitrand((N,1))
end

initialise_network (generic function with 1 method)

## Learning

In [4]:
function learning_step(
        weights,
        neuron_states,
        activations
        )
    """
    Update the firing state of one neuron randomly via 
    Hopfield's learning prescription.
    
    Args
    ----
    weights: matrix
        matrix of weights between neurons 
    
    neuron_states: vector
        vector of current binary neuron states in the
        system. 
    
    activations: vector 
        The activation threshold for each neuron.
    
    Returns 
    -------
    updated_neuron_states: vector
    """
    # Select a neuron to update 
    selection = rand(1:length(neuron_states))
    
    # Compute the input to the neuron 
    input = dot(weights[selection,:], neuron_states)

    # Update the neuron state
    if input > activations[selection]
        neuron_states[selection] = 1
    else
        neuron_states[selection] = 0
    end

    # Return the updated neuron states
    return neuron_states
end

learning_step (generic function with 1 method)

# Rough

## Learning

In [8]:
# Run test
# Have 10 neurons and 5 memories
neurons = 50
memories = 10

# Generate random memories
memories = generate_memories(neurons,memories)

# Initialise weights
weights = initialise_weights(memories)

50×50 Matrix{Int8}:
 10   2  -4   4   0   4   6   4  -2  …   2   0   2   2   2   0  -8  -2   2
  2  10  -4  -4  -4   0  -2   4  -6      2  -4   2  10  -2   4   0  -2   2
 -4  -4  10   2  -2   2  -4  -2   4     -4  -2  -4  -4  -4   2   2   4  -8
  4  -4   2  10   2   6   0  -2   0     -4   2   0  -4   0  -2  -6   0   0
  0  -4  -2   2  10   2   4  -2   0      4   2  -4  -4   0  -6  -2  -4   4
  4   0   2   6   2  10   0  -2   0  …   0  -2  -4   0   0  -2  -6  -4   0
  6  -2  -4   0   4   0  10   4   2      6   0   2  -2   2   0  -4  -2   2
  4   4  -2  -2  -2  -2   4  10  -4      0  -6   4   4   0   6  -2   4   0
 -2  -6   4   0   0   0   2  -4  10      2   4  -2  -6   2   0   4   2  -6
  0  -4   2   2  -2   2   0  -6   4      0   2   0  -4   0  -2  -2  -4   0
  0   0   2   6  -2   6  -4  -2   0  …  -4  -2   0   0   0   2  -2   0   0
 -2   2   4  -4  -4   0  -2   4  -2     -2  -8  -2   2  -2   4   0   2  -2
 -2  -2   0   0   0  -4  -2  -4   2     -2   8   2  -2   2  -4   4   2  -2
  ⋮  

In [ ]:
network = initialise_network(neurons)

# Perform learning 
steps = 100

# Activations as array of length neurons all set to 0 
activations = zeros(Int8,(neurons,1))


In [10]:
# Iterate through learning 
for ii in 1:steps
    network = learning_step(weights,network,activations)
    print('\n', network)
end

input: 22
activation: 0
Bool[0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 0; 1; 1; 1; 0; 0; 0; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 1; 0; 1; 0;;]input: 10
activation: 0
Bool[0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 0; 1; 1; 1; 0; 0; 0; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 1; 0; 1; 0;;]input: -6
activation: 0
Bool[0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 0; 1; 1; 1; 0; 0; 0; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 1; 0; 1; 0;;]input: 18
activation: 0
Bool[0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 0; 1; 1; 1; 0; 0; 0; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 1; 0; 1; 0;;]input: 10
activation: 0
Bool[0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 0; 1; 1; 1; 0; 0; 0; 0; 0; 1; 1; 1; 1; 1; 0; 1; 0; 1; 0; 1; 0;;]input: -24
activation: 0
Bool[0; 0; 0; 0; 0; 0; 1; 0; 0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 1; 0; 0; 1; 1; 1

In [11]:
# Memory recall 
# See if the network is able to stabilise on a trained memory 
# Recall the first memory
recall = memories[:,1]

network = initialise_network(neurons)

# Perform learning 
steps = 100

# Activations as array of length neurons all set to 0 
activations = zeros(Int8,(neurons,1))

50-element BitVector:
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 1
 1
 ⋮
 0
 1
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0